# **Information Retrieval and Extraction Course: Assignment 2**

## **1. Goal**

In this assignment, we'll explore three fundamental approaches to information retrieval: the **Probabilistic IR Model, Latent Semantic Indexing (LSI), and the use of BERT embeddings for similarity calculations**. These tasks aim to deepen your understanding of how to identify informative index terms (keywords) that effectively characterize a collection of documents and queries, and how to effectively represent document content for retrieval.

We'll begin by applying stemming using the Porter algorithm to reduce words to their base forms. Next, we'll extract the most informative words from the documents using the term frequency-inverse document frequency (tf-idf) weighting scheme. This step will highlight terms particularly relevant to individual documents.

Finally, we'll implement the Probabilistic IR Model, which leverages probability to assess document relevance to a query, and Latent Semantic Indexing (LSI), which aims to improve retrieval by uncovering hidden relationships between words. Additionally, we'll use BERT embeddings and calculate cosine similarity to measure the relatedness between documents and queries. This approach utilizes BERT's contextualized embeddings to enhance our ability to identify relevant documents. Through these exercises, you will gain practical experience with different information retrieval techniques.

## **2. Introduction**

### **2.1 Probabilistic Model for Information Retrieval (IR)**

The **probabilistic model** in information retrieval aims to estimate the likelihood that a document is relevant to a user's query. The goal is to approximate the ideal set of all relevant documents for a query based on the available information, primarily the index terms (keywords) present in documents and the query.
Given a user query $q$ and a document $d_j$, the model seeks to determine the probability that the user would find $d_j$ relevant. The fundamental assumption is that this probability depends solely on the query's and document's representations.

The model leverages Bayes' Rule to express the similarity between a document and a query as:
\begin{equation}
sim(d_{j},q)= \frac{P(R|d_{j})}{P(\bar{R}|d_{j})}= \frac{P(d_{j}|R)\cdot P(R)}{P(d_{j}|\bar{R})\cdot P(\bar{R})},
\end{equation}

where: 
1. $P(R|d_j)$ is the probability that $d_j$ is relevant to $q$.
2. $P(\bar{R}|d_j)$ is the probability of it being non-relevant. 
3. $P(d_j|R)$ and $P(d_j|\bar{R})$ represent the probabilities of selecting the document from the sets of relevant $(R)$ and and non-relevant $(\bar{R})$ documents, respectively.

A key simplification often made is the **independence assumption**—assuming that the presence or absence of one index term does not influence the presence or absence of other terms. While this assumption might be unrealistic, it simplifies the calculations significantly.

Under this assumption, we can approximate the similarity as:
\begin{equation}
sim(d_{j},q)=\sum_{i=1}^{k} w_{iq} \cdot w_{ij} \cdot \left( \log \frac{P(t_{i}|R)}{1-P(t_{i}|R)} +\log \frac{1-P(t_{i}|\bar{R})}{P(t_{i}|\bar{R})} \right),
\end{equation}

where:
1. $w_{iq}$ is the weight of term $t_i$ in the query $q$.
2. $w_{ij}$ is the weight of term $t_i$ in the document $d_j$.
3. $P(t_i|R)$ is the probability that index term $t_i$ is present in a relevant document.
4. $P(t_i|\bar{R})$ is the probability that index term $t_i$ is present in a non-relevant document.

Since we don't know the actual relevant set $R$ beforehand, we need a way to initialize and update these probabilities. A common initial assumption is that $P(t_i|R)$ is constant for all terms (e.g., 0.5), implying that each term is equally likely to appear in a relevant document. The probability $P(t_i|\bar{R})$ is often approximated by the distribution of the term in the entire document collection.

### Iterative Refinement


After an initial retrieval based on these assumptions, we can iteratively refine the probabilities. This process involves taking a subset 
$V$ of the top-ranked documents and updating the probabilities based on the presence or absence of terms in this subset:
\begin{equation}
P(t_{i}|R):=\frac{|V_{i}|}{|V|}, P(t_{i}|\bar{R}):=\frac{n_{i}-|V_{i}|}{N-|V|}.
\end{equation}

Here, 
1. $V_i$ represents documents in $V$ that contain term $t_i$.
2. $n_i$ is the total number of documents containing $t_i$.
3. $N$ is the total number of documents in the collection.

### Smoothing Probabilities

To address potential issues with zero probabilities (when a term is absent from the initial retrieved set), we introduce adjustment factors:
\begin{equation}
P(t_{i}|R):=\frac{|V_{i}|+0.5}{|V|+1}, P(t_{i}|\bar{R}):=\frac{n_{i}-|V_{i|}+0.5}{N-|V|+1}.
\end{equation}
These adjustments help smooth the probability estimates and prevent terms from being entirely discarded early in the retrieval process.
By iteratively updating these probabilities, the probabilistic model refines its understanding of relevance and improves its retrieval effectiveness.



### **2.2 Latent semantic indexing model**

The **Latent Semantic Indexing (LSI)** model addresses the limitations of pure keyword matching in information retrieval by capturing semantic relationships between terms and documents. Traditional keyword-based methods struggle with issues such as **synonymy** (different words with the same meaning) and polysemy (words with multiple meanings). LSI overcomes these challenges by representing documents and queries in a lower-dimensional "concept space," where semantically related items are closer together.

### Term-Document Matrix and SVD

LSI begins with a term-document matrix $M$, where where each element $M_{ij}$ represents the weight (e.g., tf-idf) of term $t_{i}$ in the document $d_{j}$. The model then applies Singular Value Decomposition (SVD) to factorize $M$.
    
\begin{equation}
M=S \cdot  \Delta \cdot D^{T}.
\end{equation}

Here: 
1. $S$ contains the eigenvectors of $M\cdot M^{T}$, representing the principal directions of variation in the term space.
2. $D$ holds the eigenvectors of $M\cdot M^{T}$, capturing variations in the document space.
3. $\Delta$ is a diagonal matrix containing singular values, which reflect the importance of each eigenvector in explaining the data. $\Delta$ is an $r \times r$ diagonal matrix of singular values where $r=min(k,N)$ is the rank of $M$.

### Dimensionality Reduction
The core of LSI is dimensionality reduction. By selecting the top $l$ largest singular values from $\Delta$ and their corresponding vectors in $S$ and $D$, we create a reduced-rank approximation $M_l$;
    
\begin{equation}
M_{l}=S_{l} \cdot  \Delta_{l} \cdot D_{l}^{T},
\end{equation}

where $l$, $l<r$, is the dimensionality of the reduced concept space. The selection of a value for $l$ tries to balance two opposing effects. It means that $l$ should be large enough to fit all  the  structure in the real data. On the other hand, it should be small enough to filter out non-relevant details of data.
The relation between any two documents in the reduced space of dimensionality $l$ can be derived from the matrix $M_{l}^{T} \cdot M_{l}$, where element $(i,j)$ quantifies the relationship between documents  $d_{i}$ and  $d_{j}$.
    
To handle queries, LSI treats a query as a pseudo-document. It is represented as a vector in the original term space and then projected into the reduced concept space using the transformation derived from SVD. Finally, document relevance is determined by calculating the cosine similarity between the query vector and each document vector in this reduced space. Documents with higher cosine similarity to the query are considered more relevant.

## **3. List of tasks**

Implement the following tasks in **sequence**. For any task that requires additional explanation, include it in your report. You may submit Jupyter notebooks along with a Report.pdf in your submission.

1. Use the data from `queries.json` (containing the queries) and `documents.json` (containing the documents), and apply tokenization to prepare them for further processing.
2. Perform stemming using the Porter algorithm.
3. Compute **term frequency** $tf_{ij}$ and $tf-idf_{ij}$ for each document with and without the removal of stop words.
4. Build a probabilistic model (experiment with different updating rules) and a latent semantic indexing (LSI) model based on the top $p$ stems according to tf-idf for each document, without removing stop words. For each query, retrieve and rank the relevant documents for both models. Compare the rankings of these documents across the two models.
5. Using the previously computed term frequency $tf_{ij}$ and $tf-idf_{ij}$ for each document after removing stop words, select the top $p$ stems according to tf-idf for each document. Build new probabilistic (experimenting with different updating rules) and LSI models using these stems. For the same $s$ queries as before, retrieve and rank the relevant documents using both models. Compare the current rankings with those obtained in Task 4.
6. **Without applying the Porter stemmer or stop word removal**, use the all-MiniLM-L6-v2 model to encode both the queries and the documents. Find the similarity between the encoded vectors and rank the documents based on this similarity. Compare these rankings with the results from the probabilistic and LSI models.


## **4. Assessment**

The assessment is based on your report and code. Your PDF report should include all experimental results, your answers to all questions, and your analysis and comments of the experimental results. Please try to detail the report by giving examples and conclusions.

**An in person evaluation will be conducted, in which you are required to walk us through your code and report.**

Please note that the deadline is **11th October 2024**, and **will not be extended.** Use moodle for all queries.

### **Mark Distribution**
-  **Data Preparation**: 5 marks
-  **Stemming and TF-IDF Calculation**: 10 marks
-  **Probabilistic Models**: 30 marks
-  **LSI Models**: 25 marks
-  **Transformer-based Model**: 15 marks 
-  **Report, Conclusions, and Comparison with Other Models**: 15 marks